In [ ]:
pip install pattern

In [ ]:
pip install newsapi-python

In [ ]:
import requests
import nltk
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import random
from wordcloud import WordCloud
import os
import spacy
nlp = spacy.load('en_core_web_sm')
from textblob import TextBlob
from newsapi import NewsApiClient
import nltk
nltk.download('omw-1.4')
from pattern.en import sentiment
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
newsapi = NewsApiClient(api_key='96e7d2a261404b4c8a2e62cfae111c1b')

In [ ]:
sources = newsapi.get_sources()
sources

In [ ]:
all_articles = newsapi.get_everything(q='narendra modi',
                                      sources='bbc-news,the-verge,abc-news,the-hindu,google-news-in,the-times-of-india,time,wired,the-wall-street-journal,techcrunch-cn,rte,business-insider,bloomberg,bbc-sport',
                                      language='en',
                                      sort_by='relevancy')
# all_articles

In [ ]:
df = pd.DataFrame.from_dict(all_articles['articles'])
colsToDrop = ['author', 'urlToImage', 'publishedAt']
train_data = df.drop(colsToDrop, axis=1)
train_data.head()


,source,title,description,url,content
0,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad Summit: World faces 'dark hour' with Ukra...,The US president met key Asia allies to discus...,https://www.bbc.co.uk/news/world-asia-61560784,By Zubaidah Abdul Jalil &amp; Tessa WongBBC Ne...
1,"{'id': 'bbc-news', 'name': 'BBC News'}",KK: 'Devastated' fans and colleagues pay tribu...,"The singer, 53, died hours after performing at...",https://www.bbc.co.uk/news/world-asia-india-61...,"By Meryl SebastianBBC News, Delhi\r\nImage sou..."
2,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad summit: The China factor at the heart of ...,The group will refocus on an open and safe Ind...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Vikas PandeyBBC News, Delhi\r\nImage source..."
3,"{'id': 'bbc-news', 'name': 'BBC News'}",Bulldozers: How a machine has become a vehicle...,Critics say India's ruling government is using...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Geeta PandeyBBC News, Delhi\r\nBulldozers, ..."
4,"{'id': 'the-times-of-india', 'name': 'The Time...",UP will drive India's growth story this centur...,​​Some memorandums of understanding (MoUs) wer...,https://economictimes.indiatimes.com/news/poli...,Prime Minister Narendra Modi on Friday said on...


In [ ]:
#Normalization

def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))

    return df
data_clean = clean_text(train_data, 'description', 'text_normalized')
data_clean.head()

,source,title,description,url,content,text_normalized
0,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad Summit: World faces 'dark hour' with Ukra...,The US president met key Asia allies to discus...,https://www.bbc.co.uk/news/world-asia-61560784,By Zubaidah Abdul Jalil &amp; Tessa WongBBC Ne...,the us president met key asia allies to discus...
1,"{'id': 'bbc-news', 'name': 'BBC News'}",KK: 'Devastated' fans and colleagues pay tribu...,"The singer, 53, died hours after performing at...",https://www.bbc.co.uk/news/world-asia-india-61...,"By Meryl SebastianBBC News, Delhi\r\nImage sou...",the singer died hours after performing at a p...
2,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad summit: The China factor at the heart of ...,The group will refocus on an open and safe Ind...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Vikas PandeyBBC News, Delhi\r\nImage source...",the group will refocus on an open and safe ind...
3,"{'id': 'bbc-news', 'name': 'BBC News'}",Bulldozers: How a machine has become a vehicle...,Critics say India's ruling government is using...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Geeta PandeyBBC News, Delhi\r\nBulldozers, ...",critics say indias ruling government is using ...
4,"{'id': 'the-times-of-india', 'name': 'The Time...",UP will drive India's growth story this centur...,​​Some memorandums of understanding (MoUs) wer...,https://economictimes.indiatimes.com/news/poli...,Prime Minister Narendra Modi on Friday said on...,some memorandums of understanding mous were al...


In [ ]:
data_clean['text_normalized'][0]

In [ ]:
#stemming

data_clean['text_stemming'] = data_clean['text_normalized'].apply(lambda x: word_tokenize(x))
data_clean.head()


,source,title,description,url,content,text_normalized,text_stemming
0,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad Summit: World faces 'dark hour' with Ukra...,The US president met key Asia allies to discus...,https://www.bbc.co.uk/news/world-asia-61560784,By Zubaidah Abdul Jalil &amp; Tessa WongBBC Ne...,the us president met key asia allies to discus...,"[the, us, president, met, key, asia, allies, t..."
1,"{'id': 'bbc-news', 'name': 'BBC News'}",KK: 'Devastated' fans and colleagues pay tribu...,"The singer, 53, died hours after performing at...",https://www.bbc.co.uk/news/world-asia-india-61...,"By Meryl SebastianBBC News, Delhi\r\nImage sou...",the singer died hours after performing at a p...,"[the, singer, died, hours, after, performing, ..."
2,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad summit: The China factor at the heart of ...,The group will refocus on an open and safe Ind...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Vikas PandeyBBC News, Delhi\r\nImage source...",the group will refocus on an open and safe ind...,"[the, group, will, refocus, on, an, open, and,..."
3,"{'id': 'bbc-news', 'name': 'BBC News'}",Bulldozers: How a machine has become a vehicle...,Critics say India's ruling government is using...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Geeta PandeyBBC News, Delhi\r\nBulldozers, ...",critics say indias ruling government is using ...,"[critics, say, indias, ruling, government, is,..."
4,"{'id': 'the-times-of-india', 'name': 'The Time...",UP will drive India's growth story this centur...,​​Some memorandums of understanding (MoUs) wer...,https://economictimes.indiatimes.com/news/poli...,Prime Minister Narendra Modi on Friday said on...,some memorandums of understanding mous were al...,"[some, memorandums, of, understanding, mous, w..."


In [ ]:
type(data_clean['text_stemming'])

pandas.core.series.Series

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text
data_clean['text_tokens_lemma'] = data_clean['text_stemming'].apply(lambda x: word_lemmatizer(x))
data_clean.head()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,source,title,description,url,content,text_normalized,text_stemming,text_tokens_lemma
0,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad Summit: World faces 'dark hour' with Ukra...,The US president met key Asia allies to discus...,https://www.bbc.co.uk/news/world-asia-61560784,By Zubaidah Abdul Jalil &amp; Tessa WongBBC Ne...,the us president met key asia allies to discus...,"[the, us, president, met, key, asia, allies, t...","[the, u, president, met, key, asia, ally, to, ..."
1,"{'id': 'bbc-news', 'name': 'BBC News'}",KK: 'Devastated' fans and colleagues pay tribu...,"The singer, 53, died hours after performing at...",https://www.bbc.co.uk/news/world-asia-india-61...,"By Meryl SebastianBBC News, Delhi\r\nImage sou...",the singer died hours after performing at a p...,"[the, singer, died, hours, after, performing, ...","[the, singer, died, hour, after, performing, a..."
2,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad summit: The China factor at the heart of ...,The group will refocus on an open and safe Ind...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Vikas PandeyBBC News, Delhi\r\nImage source...",the group will refocus on an open and safe ind...,"[the, group, will, refocus, on, an, open, and,...","[the, group, will, refocus, on, an, open, and,..."
3,"{'id': 'bbc-news', 'name': 'BBC News'}",Bulldozers: How a machine has become a vehicle...,Critics say India's ruling government is using...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Geeta PandeyBBC News, Delhi\r\nBulldozers, ...",critics say indias ruling government is using ...,"[critics, say, indias, ruling, government, is,...","[critic, say, india, ruling, government, is, u..."
4,"{'id': 'the-times-of-india', 'name': 'The Time...",UP will drive India's growth story this centur...,​​Some memorandums of understanding (MoUs) wer...,https://economictimes.indiatimes.com/news/poli...,Prime Minister Narendra Modi on Friday said on...,some memorandums of understanding mous were al...,"[some, memorandums, of, understanding, mous, w...","[some, memorandum, of, understanding, mous, we..."


In [ ]:
#stop words



In [ ]:
def word_pos_tagger(text):
    pos_tagged_text = nltk.pos_tag(text)
    return pos_tagged_text
nltk.download('averaged_perceptron_tagger')
data_clean['text_tokens_pos_tagged'] = data_clean['text_tokens_lemma'].apply(lambda x: word_pos_tagger(x))
data_clean.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,source,title,description,url,content,text_normalized,text_stemming,text_tokens_lemma,text_tokens_pos_tagged
0,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad Summit: World faces 'dark hour' with Ukra...,The US president met key Asia allies to discus...,https://www.bbc.co.uk/news/world-asia-61560784,By Zubaidah Abdul Jalil &amp; Tessa WongBBC Ne...,the us president met key asia allies to discus...,"[the, us, president, met, key, asia, allies, t...","[the, u, president, met, key, asia, ally, to, ...","[(the, DT), (u, JJ), (president, NN), (met, VB..."
1,"{'id': 'bbc-news', 'name': 'BBC News'}",KK: 'Devastated' fans and colleagues pay tribu...,"The singer, 53, died hours after performing at...",https://www.bbc.co.uk/news/world-asia-india-61...,"By Meryl SebastianBBC News, Delhi\r\nImage sou...",the singer died hours after performing at a p...,"[the, singer, died, hours, after, performing, ...","[the, singer, died, hour, after, performing, a...","[(the, DT), (singer, NN), (died, VBD), (hour, ..."
2,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad summit: The China factor at the heart of ...,The group will refocus on an open and safe Ind...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Vikas PandeyBBC News, Delhi\r\nImage source...",the group will refocus on an open and safe ind...,"[the, group, will, refocus, on, an, open, and,...","[the, group, will, refocus, on, an, open, and,...","[(the, DT), (group, NN), (will, MD), (refocus,..."
3,"{'id': 'bbc-news', 'name': 'BBC News'}",Bulldozers: How a machine has become a vehicle...,Critics say India's ruling government is using...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Geeta PandeyBBC News, Delhi\r\nBulldozers, ...",critics say indias ruling government is using ...,"[critics, say, indias, ruling, government, is,...","[critic, say, india, ruling, government, is, u...","[(critic, JJ), (say, VBP), (india, JJ), (rulin..."
4,"{'id': 'the-times-of-india', 'name': 'The Time...",UP will drive India's growth story this centur...,​​Some memorandums of understanding (MoUs) wer...,https://economictimes.indiatimes.com/news/poli...,Prime Minister Narendra Modi on Friday said on...,some memorandums of understanding mous were al...,"[some, memorandums, of, understanding, mous, w...","[some, memorandum, of, understanding, mous, we...","[(some, DT), (memorandum, NN), (of, IN), (unde..."


In [ ]:
print(type(data_clean['text_tokens_lemma']))

<class 'pandas.core.series.Series'>


In [ ]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()
results = [sia.polarity_scores(line) for line in data_clean.text_normalized]

scores_df = pd.DataFrame.from_records(results)
df = scores_df.join(data_clean, rsuffix="_right")
df.head()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,neg,neu,pos,compound,source,title,description,url,content,text_normalized,text_stemming,text_tokens_lemma,text_tokens_pos_tagged
0,0.000,1.000,0.000,0.0000,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad Summit: World faces 'dark hour' with Ukra...,The US president met key Asia allies to discus...,https://www.bbc.co.uk/news/world-asia-61560784,By Zubaidah Abdul Jalil &amp; Tessa WongBBC Ne...,the us president met key asia allies to discus...,"[the, us, president, met, key, asia, allies, t...","[the, u, president, met, key, asia, ally, to, ...","[(the, DT), (u, JJ), (president, NN), (met, VB..."
1,0.247,0.753,0.000,-0.5574,"{'id': 'bbc-news', 'name': 'BBC News'}",KK: 'Devastated' fans and colleagues pay tribu...,"The singer, 53, died hours after performing at...",https://www.bbc.co.uk/news/world-asia-india-61...,"By Meryl SebastianBBC News, Delhi\r\nImage sou...",the singer died hours after performing at a p...,"[the, singer, died, hours, after, performing, ...","[the, singer, died, hour, after, performing, a...","[(the, DT), (singer, NN), (died, VBD), (hour, ..."
2,0.000,0.854,0.146,0.4404,"{'id': 'bbc-news', 'name': 'BBC News'}",Quad summit: The China factor at the heart of ...,The group will refocus on an open and safe Ind...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Vikas PandeyBBC News, Delhi\r\nImage source...",the group will refocus on an open and safe ind...,"[the, group, will, refocus, on, an, open, and,...","[the, group, will, refocus, on, an, open, and,...","[(the, DT), (group, NN), (will, MD), (refocus,..."
3,0.250,0.750,0.000,-0.4588,"{'id': 'bbc-news', 'name': 'BBC News'}",Bulldozers: How a machine has become a vehicle...,Critics say India's ruling government is using...,https://www.bbc.co.uk/news/world-asia-india-61...,"By Geeta PandeyBBC News, Delhi\r\nBulldozers, ...",critics say indias ruling government is using ...,"[critics, say, indias, ruling, government, is,...","[critic, say, india, ruling, government, is, u...","[(critic, JJ), (say, VBP), (india, JJ), (rulin..."
4,0.000,0.866,0.134,0.4767,"{'id': 'the-times-of-india', 'name': 'The Time...",UP will drive India's growth story this centur...,​​Some memorandums of understanding (MoUs) wer...,https://economictimes.indiatimes.com/news/poli...,Prime Minister Narendra Modi on Friday said on...,some memorandums of understanding mous were al...,"[some, memorandums, of, understanding, mous, w...","[some, memorandum, of, understanding, mous, we...","[(some, DT), (memorandum, NN), (of, IN), (unde..."


In [ ]:
negative = df.iloc[df.neg.idxmax()]
neutral = df.iloc[df.neu.idxmax()]
positive = df.iloc[df.pos.idxmax()]

print(f'Most negative: {negative.text_normalized} ({negative.url})')
print(f'Most neutral: {neutral.text_normalized} ({neutral.url})')
print(f'Most positive: {positive.text_normalized} ({positive.url})')


Most negative: bombs were hurled and gunshots fired during the violence as members of a group attempted to shut shops over alleged insulting remarks made by bjp spokesperson nupur sharma during a television debate recently a senior police official said (https://economictimes.indiatimes.com/news/politics-and-nation/on-kanpur-violence-mayawati-asks-up-government-to-take-action-by-rising-above-religion-caste/articleshow/92000635.cms)
Most neutral: the us president met key asia allies to discuss chinas influence and differences over russias invasion (https://www.bbc.co.uk/news/world-asia-61560784)
Most positive: good morning its ainsley here with a special postelection edition of the newsletter were waking up this morning to the prospect of anthony albaneses labor party taking powerfor the first time since  heres everything you need to know (https://www.bloomberg.com/news/newsletters/2022-05-22/australia-news-today-australian-election-albanese-wins-morrison-steps-down)


In [ ]:
#display the result with news link



In [ ]:
#score of the model

#accuracy of the model

#word cloud